In [ ]:
pip install pyobdc

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
from collections import deque
import heapq
import time

class Program:
    def __init__(self, inst_code, brcode, quota, cat, subcat, gender, capacity, min_cutoff=0, round_no=1):
        self.inst_code = inst_code
        self.brcode = brcode
        self.capacity = capacity
        self.gender = gender
        self.quota = quota
        self.cat = cat
        self.subcat = subcat
        self.min_cutoff = min_cutoff
        self.round_no = round_no
        self.waitlist = []

    def is_eligible(self, rk):
        if self.round_no ==1  :
            return True
        return rk <= self.min_cutoff

    def add_candidate(self, rk, roll_no):
        # Only add if eligible by rank
        # if self.is_eligible(rk):
        #     heapq.heappush(self.waitlist, (-rk, roll_no))
        #     return True
        # return False
        heapq.heappush(self.waitlist, (-rk, roll_no))
        return True
    

    def remove_least_preferred(self):
        # Remove the candidate with the highest positive rank (least preferred)
        return heapq.heappop(self.waitlist)

class Candidate:
    def __init__(self, roll_no, adv_rank, mains_rank):
        self.roll_no = roll_no
        self.adv = adv_rank
        self.mains = mains_rank
        self.current_index = 1  # Initialize to 1 assuming choice numbers start from 1
        self.max_index = 0
        self.choice_list = []
        
# Load data from CSV files
def load_candidates(filename):
    df = pd.read_csv(filename)
    filtered_df = df[(df['Adv_DS'] == 2) & 
                     (df['Nationality'] == 1)].sort_values('Adv_CRL_Rank').head(100)
    return {row['RollNo']: Candidate(row['RollNo'], row['Adv_CRL_Rank'], row['AI_Eng_CRL_Rank']) for index, row in filtered_df.iterrows()}


def load_programs(filename):
    df = pd.read_csv(filename)
    return {(row['InstCode'], row['Brcode'], row['Quota'], row['Cat'], row['SubCat'], row['Gender']): Program(row['InstCode'], row['Brcode'], row['Quota'], row['Cat'], row['SubCat'], row['Gender'], row['TotalSeat']) for index, row in df.iterrows()}

def load_choices(filename):
    df = pd.read_csv(filename)
    for row in df.iterrows():
        roll_no = row['RollNO']
        choice_no = row['ChoiceNo']
        candidates[roll_no].max_index+=1
        
        inst_code = row['InstCode']
        brcode = row['Brcode']
        quota = row['AllottedQuota']
        cat = row['AllottedCategory'][:2]
        subcat = row['AllottedCategory'][2:]
        gender = row['AllottedGender']
        
        program_key = (inst_code, brcode, quota, cat, subcat, gender)
        program = programs.get(program_key)
        candidates[roll_no].choice_list[choice_no-1]=program


# Example file paths
candidates = load_candidates(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round1\Candidate.csv')
programs = load_programs(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round1\Seats.csv')
choices = load_choices(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round1\Choice.csv')

# Function to get preferences based on Choices DataFrame
def get_preference(roll_no, choice_no, choices_df):
    row = choices_df[(choices_df['RollNo'] == roll_no) & (choices_df['ChoiceNo'] == choice_no)]
    if not row.empty:
        inst_code, br_code = row.iloc[0]['InstCode'], row.iloc[0]['Brcode']
        return programs.get((inst_code, br_code, 'AI', 'OP', 'NO', 'B'))
    return None

# Processing candidates
queue = deque(sorted(candidates.values(), key=lambda c: c.adv))  # Sort by advanced rank for processing
# queue = deque([candidates[roll_no] for roll_no in candidates])

def process_candidates():
    start_time = time.time()
    allocations = {}
    while queue:
        candidate = queue.popleft()
        preference = get_preference(candidate.roll_no, candidate.current_index, choices)
        if preference:
            rk = candidates[candidate.roll_no].adv if inst_code < 200 else candidates[candidate.roll_no].mains
            if preference.add_candidate(rk, candidate.roll_no):
                allocations[candidate.roll_no] = (preference.inst_code, preference.brcode)
                if len(preference.waitlist) > preference.capacity:
                    last_rank = -preference.waitlist[0][0]  # Assuming waitlist stores (-rank, roll_no)
                    if last_rank > preference.min_cutoff:
                        # Count how many have the same rank as the last rank
                        count_same_rank = sum(1 for x in preference.waitlist if -x[0] == last_rank)

                        # Check if removing these candidates brings us below capacity
                        if len(preference.waitlist) - count_same_rank >= preference.capacity:
                            removed_candidates = []

                            # Remove all candidates with the least preferred rank
                            while preference.waitlist and -preference.waitlist[0][0] == last_rank:
                                _, roll_no = heapq.heappop(preference.waitlist)  # Pop the least preferred candidate
                                removed_candidates.append(roll_no)

                            # Requeue the removed candidates for further processing
                            for roll_no in removed_candidates:
                                candidates[roll_no].current_index += 1  # Move to next preference
                                candidate = candidates[roll_no] 
                                queue.append(candidate)  # Requeue for another attempt
            else:
                candidates[candidate.roll_no].current_index += 1
                queue.append(candidate)  # Add back to the queue if not placed  
                
    print(f"Time taken: {time.time() - start_time:.2f} seconds")
    return allocations, programs

allocations , final_programs = process_candidates()
 j

print("Final Allocations:")
for roll_no, (inst_code, brcode) in allocations.items():
    candidate = candidates.get(roll_no)
    print(f"Candidate {roll_no} Rank - {candidate.adv}  Choice - {candidate.current_index} - is allocated to program {inst_code}'s {brcode}")
  
print("\nFinal Waitlists:")
for program_key, program in final_programs.items():
    if program.waitlist:
        print(f"Program {program_key} waitlist: {program.waitlist} with {program.capacity - len(program.waitlist)} remaining seats")
    
    
print("\nFinal Cutoffs:")
for program_key, program in final_programs.items():
    if program.waitlist:
        last_rank = -program.waitlist[0][0]
        print(f"Program {program_key} cutoff: {last_rank}")